In [ ]:
!pip install snscrape tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 14.0 MB/s eta 0:00:00


**Notebook do Thiago para fazer o scrapping, mas vou pegar somente o tweeet e não as respostas.**

In [ ]:
from dataclasses import fields
from datetime import datetime
import pandas as pd
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm

# Oculta warnings do notebook
# import warnings
# warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 80)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Começamos filtrando todos os tweets de um usuario, dentro do período desejado
username = 'ErikakHilton'  # exemplo
since = '2022-10-03'
until = '2022-10-04'

query = f'from:{username} since:{since} until:{until}'
user_scrapping_results = sntwitter.TwitterSearchScraper(query)

tweet_ids = []
for tweet in tqdm(user_scrapping_results.get_items()):
    tweet_ids.append(tweet.id)

print(len(tweet_ids))

13it [00:00, 13.89it/s]

13


In [ ]:
# Vamos iterar entre os id's para pegar os dados dos tweets
tweets = []

for tweet_id in tqdm(tweet_ids):
    tweet_scrapper = sntwitter.TwitterTweetScraper(tweet_id, mode=sntwitter.TwitterTweetScraperMode.SINGLE)
    for tweet in tweet_scrapper.get_items():
        tweets.append(tweet)
        
print(len(tweets))

100%|██████████| 13/13 [00:04<00:00,  2.80it/s]

13


In [ ]:
tweets

[Tweet(url='https://twitter.com/ErikakHilton/status/1577082819133984768', date=datetime.datetime(2022, 10, 3, 23, 46, 59, tzinfo=datetime.timezone.utc), rawContent='@pedroneschling Obrigada, querido! 💖', renderedContent='@pedroneschling Obrigada, querido! 💖', id=1577082819133984768, user=User(username='ErikakHilton', id=738143559920934912, displayname='ERIKA HILTON 🚩 🇧🇷', rawDescription='Deputada Federal - PSOL/SP (2023-2026)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.', renderedDescription='Deputada Federal - PSOL/SP (2023-2026)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.', descriptionLinks=None, verified=True, created=datetime.datetime(2016, 6, 1, 23, 1, 53, tzinfo=datetime.timezone.utc), followersCount=265395, friendsCount=4795, statusesCount=26992, favouritesCount=65711, listedCount=476, mediaCount=3400, location='São Paulo, Brasil', protected=False, link=Te

In [ ]:
# Convertendo as informações para DataFrame
tweet_fields = [f.name for f in fields(tweet)]
user_fields = [f.name for f in fields(tweet.user)]
all_columns = tweet_fields + ['user_' + f for f in user_fields]

tweets_data = [
    [getattr(t, field) for field in tweet_fields] + [getattr(t.user, field) for field in user_fields] 
    for t in tweets
    ]
    
tweets_df = pd.DataFrame(tweets_data, columns=all_columns)

print(tweets_df.shape)

(13, 49)


In [ ]:
pd.options.display.max_colwidth = 1000

In [ ]:
tweets_df

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe,user_username,user_id,user_displayname,user_rawDescription,user_renderedDescription,user_descriptionLinks,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_link,user_profileImageUrl,user_profileBannerUrl,user_label
0,https://twitter.com/ErikakHilton/status/1577082819133984768,2022-10-03 23:46:59+00:00,"@pedroneschling Obrigada, querido! 💖","@pedroneschling Obrigada, querido! 💖",1577082819133984768,https://twitter.com/ErikakHilton,8,1,314,0,1577070240524996608,pt,None,None,None,None,None,None,None,"1,577,070,240,524,996,608.00",https://twitter.com/pedroneschling,[https://twitter.com/pedroneschling],None,None,None,None,None,None,None,ErikakHilton,738143559920934912,ERIKA HILTON 🚩 🇧🇷,"Deputada Federal - PSOL/SP (2023-2026)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.","Deputada Federal - PSOL/SP (2023-2026)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.",None,True,2016-06-01 23:01:53+00:00,265395,4795,26992,65711,476,3400,"São Paulo, Brasil",False,"TextLink(text='erikahilton.com.br', url='http://erikahilton.com.br/', tcourl='https://t.co/5KKZ3fVaUj', indices=(0, 23))",https://pbs.twimg.com/profile_images/1559380990333706240/vEImKRHW_normal.jpg,https://pbs.twimg.com/profile_banners/738143559920934912/1670375803,None
1,https://twitter.com/ErikakHilton/status/1577057162471567360,2022-10-03 22:05:02+00:00,"Obrigada pelo texto, querida @AHelenaVieira ✨\nhttps://t.co/aCZGv8Lfjy","Obrigada pelo texto, querida @AHelenaVieira ✨\nharpersbazaar.uol.com.br/estilo-de-vida…",1577057162471567360,https://twitter.com/ErikakHilton,5,14,393,3,1577057162471567360,pt,None,None,None,"[TextLink(text='harpersbazaar.uol.com.br/estilo-de-vida…', url='https://harpersbazaar.uol.com.br/estilo-de-vida/helena-vieira-o-brasil-que-vira-as-travestis-e-transexuais-ocupam-o-congresso-nacional/', tcourl='https://t.co/aCZGv8Lfjy', indices=(46, 69))]",None,None,None,NaN,None,[https://twitter.com/AHelenaVieira],None,None,None,None,"SummaryCard(title='Helena Vieira - O Brasil que virá: as travestis e transexuais ocupam o Congresso Nacional -...', url='https://harpersbazaar.uol.com.br/estilo-de-vida/helena-vieira-o-brasil-que-vira-as-travestis-e-transexuais-ocupam-o-congresso-nacional/', description='Helena Vieira - O Brasil que virá: as travestis e transexuais ocupam o Congresso Nacional após as eleições de 2022', thumbnailUrl='https://pbs.twimg.com/card_img/1646939104574341120/rpAZpGt3?format=jpg&name=orig', siteUser=User(username='BazaarBR', id=388911506, displayname=""Harper's Bazaar"", rawDescription='Moda, beleza, cultura, viagens, noivas e estilo de vida', renderedDescription='Moda, beleza, cultura, viagens, noivas e estilo de vida', descriptionLinks=None, verified=False, created=datetime.datetime(2011, 10, 11, 15, 2, 28, tzinfo=datetime.timezone.utc), followersCount=116271, friendsCount=508, statusesCount=39899, favouritesCount=136, listedCount=150, mediaCount=22677, location='São Paulo', protected=False,...",None,None,ErikakHilton,738143559920934912,ERIKA HILTON 🚩 🇧🇷,"Deputada Federal - PSOL/SP (2023-2026)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.","Deputada Federal - PSOL/SP (2023-2026)\nVereadora + votada do Brasil em 2020, presidenta duas vezes da Comissão de Direitos Humanos de São Paulo.",None,True,2016-06-01 23:01:53+00:00,265395,4795,26992,65711,476,3400,"São Paulo, Brasil",False,"TextLink(text='erikahilton.com.br', url='http://erikahilton.com.br/', tcourl='https://t.co/5KKZ3fV

In [ ]:
#Estamos interessados na coluna rended Content para a análise de sentimentos
textos_gpt = tweets_df['renderedContent'].to_string(index=False).strip().replace('\n',';;')

In [ ]:
textos_gpt

'@pedroneschling Obrigada, querido! 💖;;                                                                                                                                                                                                        Obrigada pelo texto, querida @AHelenaVieira ✨\\nharpersbazaar.uol.com.br/estilo-de-vida…;;"Super importante que conseguimos ter elegido uma mulher negra, uma indígena por São Paulo, travestis pelo Brasil e essa ocupação que a gente acredita. Esse projeto de poder que a gente está construindo!". Um pouco da minha entrevista para a @CBNoficial, assista! https://t.co/QR4X2slrx9;;                                                                                    Estamos fazendo história! @folha impressa de hj.\\n\\nMas esse novo capítulo do Brasil só terá um final feliz quando elegermos @lulaoficial nosso presidente e derrotarmos Bolsonaro! https://t.co/jaL5T3AQre;;                                                                                           

In [ ]:
import os
import openai

#Site para criar a chave: https://platform.openai.com/account/api-keys

openai.api_key = ''
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": f"Realize uma análise de sentimentos em uma série de tweets que vou mandar, classificando cada um como positivo, neutro ou negativo. Sua resposta deve vir em uma tabela.csv no seguinte formato: A primeira coluna deve conter os primeiros 20 caracteres do tweet, a segunda coluna deve conter o sentimento e a terceira coluna deve conter seus comentários sobre o tweet, sendo uma linha por tweet. Os tweets estão separados por ;;, seguem os tweets: {textos_gpt}"}
  ]
)

print(completion.choices[0].message.content)

Primeiros 20 caracteres do tweet;Sentimento;Comentários
@pedroneschling         ;positivo;Agradecimento
Obrigada pelo texto     ;positivo;Agradecimento
"Super importante que   ;positivo;Entusiasmo com entrevista
Estamos fazendo história;positivo;Entusiasmo com a vitória
@emischneiderv         ;positivo;Agradecimento pelo apoio
@RobeyonceLima         ;positivo;Agradecimento pela parceria
Pessoal!               ;positivo;Agradecimento pelas mensagens
@Mikalins              ;positivo;Agradecimento
Vamos ocupar!           ;positivo;Entusiasmo para ocupação
E, sem sombra de dúvidas;positivo;Foco na eleição de Haddad
@avee_killer           ;neutro;Concordância
@atlantico05           ;neutro;Concordância
É confirmado, teremos  ;neutro;Foco na campanha para Lula


In [ ]:
completion

<OpenAIObject chat.completion id=chatcmpl-76MPwLTgiQemhyS7ihnzK8TZhatyJ at 0x7f046231fd10> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Primeiros 20 caracteres do tweet;Sentimento;Coment\u00e1rios\n@pedroneschling         ;positivo;Agradecimento\nObrigada pelo texto     ;positivo;Agradecimento\n\"Super importante que   ;positivo;Entusiasmo com entrevista\nEstamos fazendo hist\u00f3ria;positivo;Entusiasmo com a vit\u00f3ria\n@emischneiderv         ;positivo;Agradecimento pelo apoio\n@RobeyonceLima         ;positivo;Agradecimento pela parceria\nPessoal!               ;positivo;Agradecimento pelas mensagens\n@Mikalins              ;positivo;Agradecimento\nVamos ocupar!           ;positivo;Entusiasmo para ocupa\u00e7\u00e3o\nE, sem sombra de d\u00favidas;positivo;Foco na elei\u00e7\u00e3o de Haddad\n@avee_killer           ;neutro;Concord\u00e2ncia\n@atlantico05           ;neutro;Concord\u00e2ncia\n\u00c9 confirmado, te

In [ ]:
import io
import csv
data=completion.choices[0].message.content
df=pd.read_csv(io.StringIO(data), sep=";", quoting=csv.QUOTE_NONE)

In [ ]:
df

,Primeiros 20 caracteres do tweet,Sentimento,Comentários
0,@pedroneschling,positivo,Agradecimento
1,Obrigada pelo texto,positivo,Agradecimento
2,"""Super importante que",positivo,Entusiasmo com entrevista
3,Estamos fazendo história,positivo,Entusiasmo com a vitória
4,@emischneiderv,positivo,Agradecimento pelo apoio
5,@RobeyonceLima,positivo,Agradecimento pela parceria
6,Pessoal!,positivo,Agradecimento pelas mensagens
7,@Mikalins,positivo,Agradecimento
8,Vamos ocupar!,positivo,Entusiasmo para ocupação
9,"E, sem sombra de dúvidas",positivo,Foco na eleição de Haddad


In [ ]:
df.to_csv(r'TestGPT.csv', index=False)

In [ ]:
from google.colab import files
files.download('TestGPT.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>